# Homework 1 - Streaming (15 pts)

### DUE: 02/18/2020 at 5:30pm

This homework is for practicing Python’s *generators* and the design of *streaming* algorithms in general. We’re going to use the **taxi_events.csv** and **taxi_trips.csv** data sets. Both of them are available on NYU Classes under *Data Sets* section. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW1\_NetID.ipynb.

## Task 1 (8 points)

Your task is to **compute the maximum number of active taxi cabs** that were hired at any point in time. This the same as computing the maximum number of taxi cabs that have passengers. The input data set is **taxi_events.csv**, which logged all pick-up and drop-off events for all taxi trips. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|vehicle_id |The unique ID of the taxi vehicle involved in this event. |
|event |A string of either *"pickup"* or *"dropoff"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a taxi that picked up a passenger at midnight and dropped her off at 14 minutes past midnight. If the taxi cab has the Vehicle ID of *V102*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,V102,pickup
...
2015-02-01 00:14:00+00,V102,dropoff
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of hired taxi cabs at any point in time. Again, this is equivalent to having a virtual dispatcher, who repeatedly ask every second *"how many taxis are being hired (having passengers) at the moment?"*, and then log the maximum number during the entire period.

Please modify the code snippet below to complete this task. Your code should only output a single number, which is the maximum number of hired taxi cabs. Of course, you can add additional initialization codes outside of the for loop as needed. Additional, please provide a brief rationale and/or justification for your design after the code.

In [1]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

maxHiredCabs = 0
currentlyHired = 0

for row in csvRows('taxi_events.csv'):
    if row['event'] == 'pickup':
        currentlyHired += 1
        if currentlyHired > maxHiredCabs:
            maxHiredCabs = currentlyHired
    elif row['event'] == 'dropoff':
        currentlyHired -= 1   
    
print("Max Number of Hired Cabs: ", maxHiredCabs)

Max Number of Hired Cabs:  250


#### RATIONALE AND JUSTIFICATION

**Since we're only interested in keeping track of hired cabs and the max number of concurrently hired cabs 
 we can just keep a counter that increases when a cab gets hired, and decreases when it makes a dropoff.**

**Finally to find the max number of hired cabs we check the counter against the maxHiredCabs value
 if current number, if its greater than maxHiredCabs, then we assign maxHiredCabs with the current count, else nothing happens**

## Task 2 (7 points)

The objective of this task is identical to Task 1's but you are asked to use the full **taxi_trips.csv** data set instead of the events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented in a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

Below is the description of the **taxi_trips.csv** file, which is sorted only by the pick-up time:

|Column name|Description|
|--|--|
|trip_duration |The duration of the trip in seconds. This field is for your convenience since it can be derived also from the pick-up and drop-off times. |
|pickup_time |The timestamp of the pick-up of the trip. All trip records are sorted increasingly by their pick-up times. |
|dropoff_time |The timestamp of the drop-off event. |
|vehicle_id |The unique ID of the taxi vehicle involved in this trip record. |



In [2]:
import csv
import dateutil
import time 

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

def is_overlapping(time1, time2):
    return max(time1[1], time2[1]) - min(time1[0], time2[0]) < ((time1[1]-time1[0]) + (time2[1]-time2[0])) 
    
# #First attempt
# maxHiredCabs = 0
# active = []
# start = time.time()
# for row in csvRows('taxi_trips.csv'):
#     cab_time = (dateutil.parser.parse(row['pickup_time']), 
#                 dateutil.parser.parse(row['dropoff_time']))
#     active = [item for item in active if is_overlapping(item, cab_time)]
#     active.append(cab_time)    
#     if (len(active) > maxHiredCabs):
#         maxHiredCabs = len(active)
# end = time.time()
# print(maxHiredCabs)
# print("Elapsed Time: ", end - start)

##Second attempt
endTimes = []
maxHiredCabs = 0
start = time.time()
for row in csvRows('taxi_trips.csv'):
    pickup = dateutil.parser.parse(row['pickup_time']) 
    dropoff= dateutil.parser.parse(row['dropoff_time'])

    endTimes = [endTime for endTime in endTimes if endTime > pickup ]
    endTimes.append(dropoff)
    maxHiredCabs = max(maxHiredCabs, len(endTimes))

end = time.time()
print(maxHiredCabs)
print("Elapsed Time: ", end - start)

250
Elapsed Time:  21.345616579055786


#### RATIONALE AND JUSTIFICATION

*Please explain your solution here*

**In my first attempt I'm keeping an array of all intervals that overlap, when a new interval comes in we filter out all intervals that do not overlap with the new interval. Then to get the current number of hired cars, we just take the size of the array that has all these overlapping intervals.**

**In my second attempt I realized that you only need to store drop off times. When a new time interval comes in, remove any drop_off time from the list that is less the new interval's start time because these dropoff times have finished and are no longer relevant/overlapping. After filtering add the end time to the list. Then to keep track of max hired cabs we just check the length of the list vs the previous value of maxHiredCabs, and keep the biggest.**